In [1]:
from abtem import __version__
print('current version:', __version__)

import matplotlib.pyplot as plt
from ase.build import mx2

from abtem import *
from abtem.structures import orthogonalize_cell

from abtem.visualize.interactive.apps import ArrayView1d, MeasurementView2d
from abtem.visualize.interactive.tools import PanZoomTool, BoxZoomTool, SelectPixelTool
from abtem.visualize.interactive.artists import ImageArtist, ColorBar
from abtem.visualize.interactive.utils import throttle
import ipywidgets as widgets
from traitlets import link
import numpy as np
from tqdm.auto import tqdm

current version: 1.0.0beta20


# Exploring 4D-STEM dataset

In [2]:
atoms = mx2(formula='MoS2', kind='2H', a=3.18, thickness=3.19, size=(1, 1, 1), vacuum=None)

repetitions = (7, 4, 1)

atoms = orthogonalize_cell(atoms)

atoms *= repetitions

atoms.center(vacuum=2, axis=2)

atoms *= (1,1,3)

In [3]:
potential = Potential(atoms, 
                      gpts=512,
                      projection='finite', 
                      slice_thickness=1, 
                      parametrization='kirkland').build(pbar=False)


S = SMatrix(energy=80e3, semiangle_cutoff=20, expansion_cutoff=20, device='cpu', rolloff=0.1)

S.grid.match(potential)

detector = PixelatedDetector()

scan_end = (potential.extent[0] / repetitions[0], potential.extent[1] / repetitions[1])
gridscan = GridScan(start=[0, 0], end=scan_end, sampling=S.ctf.nyquist_sampling * .25)

measurement = S.scan(gridscan, detector, potential, pbar=False)

annular_detector = AnnularDetector(40,100)
annular_measurement = annular_detector.integrate(measurement)

In [4]:
view1 = MeasurementView2d(annular_measurement)
view2 = MeasurementView2d(measurement, navigation_axes=(0,1))

view1.canvas.tools = {'Select':SelectPixelTool(view1._array_view._image_artist)}
view2.canvas.tools = {'PanZoom':PanZoomTool(), 'BoxZoom':BoxZoomTool()}

link((view1.canvas.tools['Select'], 'index_x'), (view2._array_view._sliders[0], 'value'))
link((view1.canvas.tools['Select'], 'index_y'), (view2._array_view._sliders[1], 'value'))

view1.artist.color_bar.label = 'Intensity [arb. unit]'
view2.artist.color_bar.label = 'Intensity [arb. unit]'

widgets.HBox([widgets.VBox([view1.figure, view1.artist.color_bar, 
                            view1.canvas.toolbar, view1.artist.color_scheme_picker]),
              
              widgets.VBox([view2.figure, view2.artist.color_bar, 
                            view2.canvas.toolbar, view2.artist.color_scheme_picker]), ])

In [5]:
view2.artist.power_scale_slider

FloatSlider(value=1.0, description='Power', max=1.0, min=0.001, step=0.001)